In [32]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
gbar=9.81
L=1
Δ = 0.01
θd = np.pi/6
θ0 = 0-θd
ω0 = 0
tf=20
NN=tf / Δ
α = 0.005
β = 1
γ = 0.9
R = .05
kωe = 0.2
q = [[1, 0], [0, 1]]
σν = 0.1
kθ = 2*gbar/L
kω = 2

def phi_basis(X1, X2):
    result = np.array([[X1, X2, X1*X1, X1*X2, X2*X2]]).T
    
    return result

def D_phi_basis(X1, X2):
    result = np.array([[1, 0, 2*X1, X2, 0],
                       [0, 1, 0, X1, 2*X2]]).T
    return (result)
    
NL = 5

W0 = np.array([[np.random.normal(0, 0.1),
                np.random.normal(0, 0.1),
                np.random.normal(0, 0.1),
                np.random.normal(0, 0.1),
                np.random.normal(0, 0.1)]]).T

U0 = np.array([[np.random.normal(0, 0.1),
                np.random.normal(0, 0.1),
                np.random.normal(0, 0.1),
                np.random.normal(0, 0.1),
                np.random.normal(0, 0.1)]]).T
def f(X1, X2):
    result_part1 = np.array([[X1],[X2]])
    
    result_part2 = np.array([[X2], [((gbar/L)*(np.sin(θd)-np.sin(θd+X1))) - kωe*(X2)]])
    
    return result_part1 + Δ*result_part2

def g(u):
    result = u*np.array([0,Δ])
    return result

def r(X1, X2, u):
    X = np.array([[X1],
                  [X2]])
    result = X.T @ q @ X + R*u*u
    return float(result)

X_list = []
U_list = []
W_list = [W0]
u_list = []
u_k=1
W_k = W0
U_k = U0
kt = 0
tΔ = [0]
uj = (-γ / 2)* (1/R) * g(u_k) @ D_phi_basis(θ0, ω0).T @ W0 
Wji = W0

tc = []
θ0_list, ω0_list = [],[]

w1_list, w2_list, w3_list, w4_list, w5_list = [],[],[],[],[]


for k in range(int(NN)):
    θ0_list.append(θ0)
    ω0_list.append(ω0)
    X_k = f(θ0, ω0) + g(uj)
    X_list.append(X_k)
    θ0, ω0  = X_k[0][0], X_k[1][0]
    
    phi = phi_basis(θ0,ω0)
    
    r_k = r(θ0, ω0, u_k) + γ*float(phi_basis(θ0, ω0).T @ Wji)
    
    ecount = 1
    Wjim = W_list[-1]
    Wji = Wji - α*phi_basis(θ0, ω0)*(float(phi_basis(θ0, ω0).T @ Wji) - r_k)
    
    while ecount < 100 and float((Wji - Wjim).T @ (Wji - Wjim)) > 10**-5:
        Wjim = W_list[-1]
        #gradient descent
        Wji = Wji - α*phi_basis(θ0, ω0)*(float(phi_basis(θ0, ω0).T @ Wji) - r_k)
        ecount+=1
        
        
    W_list.append(Wji)
    
    w1_list.append(Wji[0][0])
    w2_list.append(Wji[1][0])
    w3_list.append(Wji[2][0])
    w4_list.append(Wji[3][0])
    w5_list.append(Wji[4][0])
    
    #need to update u with the weights from W
    #policy improvement - update u_j
    uj = -(γ/2)*(R**-1)*float(g(uj).T @ D_phi_basis(θ0 , ω0).T @ W_list[-1])
    u_list.append(uj)
    tc.append(k)


